# Homework 9: Linear Regression

**Reading**: 
* [Linear Regression](https://www.inferentialthinking.com/chapters/15/2/Regression_Line.html)
* [Method of Least Squares](https://www.inferentialthinking.com/chapters/15/3/Method_of_Least_Squares.html)
* [Least Squares Regression](https://www.inferentialthinking.com/chapters/15/4/Least_Squares_Regression.html)

Please complete this notebook by filling in the cells provided. Before you begin, execute the following cell to load the provided tests. Each time you start your server, you will need to execute this cell again to load the tests.

Directly sharing answers is not okay, but discussing problems with the course staff or with other students is encouraged. Refer to the policies page to learn more about how to learn cooperatively.

For all problems that you must write our explanations and sentences for, you **must** provide your answer in the designated space. Moreover, throughout this homework and all future ones, please be sure to not re-assign variables throughout the notebook! For example, if you use `max_temperature` in your answer to one question, do not reassign it later on.

In [ ]:
# Don't change this cell; just run it. 

import numpy as np
from datascience import *

# These lines do some fancy plotting magic.
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import warnings
warnings.simplefilter('ignore', FutureWarning)

import scipy.stats as stats



## Exploring the PTEN Gene with Linear Regression

## 1. PTEN Linear Regression

This week's homework is about linear regression. The dataset we'll be using is from the Cancer Cell Line Encyclopedia -- you can read more about this database in this [paper](https://www.nature.com/articles/s41586-019-1186-3) and interact with the data yourself at the online portal [here](https://portals.broadinstitute.org/ccle).

The specific dataset we'll be taking a look at is expression data for the PTEN gene in around 1000 cell lines. The PTEN gene is a tumor-suppressing gene, and mutations in the PTEN gene are associated with many types of cancer. A cell line is group of cells that are kept alive and replicate indefinitely in culture (grown in petri dishes, for example).

Run the following cell to load the `pten` table. The `pten` table has four columns, a column for the specific `Cell Line`, a column for the `Copy Number`, which is how many times a copy of a portion of the PTEN gene is found in the DNA of that cell line, `mRNA Expression (Affy)`, and `mRNA Expression (RNAseq)`.

In [ ]:
# Just run this cell
pten = Table().read_table("pten.csv")
pten.show(5)

In [ ]:
# Just run this cell
pten.hist("Copy Number", bins = np.arange(-1, 1.5, 0.5))

**Question 1**

Looking at the histogram above, we want to check whether or not `Copy Number` is in standard units. For this question, we computed the mean and the standard deviation of the values in `Copy Number` and assign these values to `copy_number_mean` and `copy_number_sd` respectively.  From these, you should be able to tell if the data is in standard units.   Assign `is_su` to either `True` if you think that `Copy Numbers` is in standard units or `False` if you think otherwise.

<!--
BEGIN QUESTION
name: q1_1
manual: false
-->

In [ ]:
copy_number = pten.column("Copy Number")
copy_number_mean = np.mean(pten.column("Copy Number"))
copy_number_sd = stats.tstd(pten.column("Copy Number"))


is_su = ...
print(f"Mean: {copy_number_mean}, SD: {copy_number_sd}, Is in standard units?: {is_su}")

**Question 2**

We're trying to create the function `standard_units` so that it converts the values in the array `arr` to standard units. We'll then use `standard_units` to create a new table, `pten_su`, that converts all the values in the table `pten` to standard units.  Based on what you know about standard units, fill in what's missing for this function.  

<!--
BEGIN QUESTION
name: q1_2
manual: false
-->

In [ ]:
def standard_units(arr):
    mean = ...
    std = ...
    return (arr - mean)/std

# DON'T DELETE OR MODIFY ANY OF THE LINES OF CODE BELOW IN THIS CELL
pten_su = Table().with_columns("Cell Line", pten.column("Cell Line"),
                               "Copy Number SU", standard_units(pten.column("Copy Number")),
                               "mRNA Expression (Affy) SU", standard_units(pten.column("mRNA Expression (Affy)")),
                               "mRNA Expression (RNAseq) SU", standard_units(pten.column("mRNA Expression (RNAseq)"))                             
                              )
pten_su.show(5)

You should always visually inspect your data, before numerically analyzing any relationships in your dataset. Run the following cell in order to look at the relationship between the variables in our dataset.

In [ ]:
# Just run this cell
pten_su.scatter("Copy Number SU", "mRNA Expression (Affy) SU")
pten_su.scatter("Copy Number SU", "mRNA Expression (RNAseq) SU")
pten_su.scatter("mRNA Expression (Affy) SU", "mRNA Expression (RNAseq) SU")

**Question 3**

Which of the following relationships do you think has the highest correlation (i.e. highest absolute value of `r`)? Assign `highest_correlation` to the number corresponding to the relationship you think has the highest correlation.

1. Copy Number vs. mRNA Expression (Affy)
2. Copy Number vs. mRNA Expression (RNAseq)
3. mRNA Expression (Affy) vs. mRNA Expression (RNAseq)

<!--
BEGIN QUESTION
name: q1_3
manual: false
-->

In [ ]:
highest_correlation = ...

**Question 4**

Now, using the `standard units` function, define the function `correlation` which computes the correlation between `array1` and `array2`.


$$r = \frac{1}{n-1} \sum_{i=1}^n \left(\frac{x_i-\overline{x}}{S_x}\right)\left(\frac{y_i-\overline{y}}{S_y}\right)$$


In [ ]:
def correlation(array1, array2):
    array1_in_su = ...
    array2_in_su = ...
    correlation = sum(array1_in_su * array2_in_su)/ (len(array1) - 1)
    return sum(standard_units(arr1)*standard_units(arr2))/(len(arr1)-1)

# This computes the correlation between the different variables in pten
copy_affy = correlation(pten.column("Copy Number"), pten.column("mRNA Expression (Affy)"))
copy_rnaseq = correlation(pten.column("Copy Number"), pten.column("mRNA Expression (RNAseq)"))
affy_rnaseq = correlation(pten.column("mRNA Expression (Affy)"), pten.column("mRNA Expression (RNAseq)"))

print(f" \
      Copy Number vs. mRNA Expression (Affy) Correlation: {copy_affy}, \n \
      Copy Number vs. mRNA Expression (RNAseq) Correlation: {copy_rnaseq}, \n \
      mRNA Expression (Affy) vs. mRNA Expression (RNAseq) Correlation: {affy_rnaseq}")

**Question 5**

If we switch what we input as arguments to `correlation`, i.e. found the correlation between `mRNA Expression (Affy)` vs. `Copy Number` instead of the other way around, would the correlation change? Assign `correlation_change` to either `True` if you think yes, or `False` if you think no.

<!--
BEGIN QUESTION
name: q1_5
manual: false
-->

In [ ]:
correlation_change = ...
correlation_change

**Question 6**

Looking at both the scatter plots after Question 2 and the correlations computed in Question 4, describe a pattern you see in the relationships between the variables.

<!--
BEGIN QUESTION
name: q1_6
manual: true
-->
<!-- EXPORT TO PDF -->

*Write your answer here, replacing this text.*

**Question 7**

Let's look at the relationship between mRNA Expression (Affy) vs. mRNA Expression (RNAseq) only. We'll use these two facts about the regression line to determine its equation.  

*Fact 1:* 

$$ b_1 = r \frac{S_y}{S_x}$$

*Fact 2:*

$$\overline{y} = b_1\cdot \overline{x} + b_0$$


We'll use the `correlation` function that you defined earlier.  To complete this problem, you need to fill in the calculations for the means, standard devations and correlation.  

In [ ]:
Sx = ...
Sy = ...

xbar = ...
ybar = ...


cor = correlation(..., ...)

slope = cor * Sy / Sx

intercept = ybar - slope * xbar

parameters = make_array(intercept, slope)

parameters

**Question 8**

If we switch what we input as arguments to `regression_parameters`, i.e. found the parameters for the regression line for `mRNA Expression (RNAseq)` vs. `mRNA Expression (Affy)` instead of the other way around, would the regression parameters change (would the slope and/or intercept change)? Assign `parameters_change` to either `True` if you think yes, or `False` if you think no.

<!--
BEGIN QUESTION
name: q1_8
manual: false
-->

In [ ]:
parameters_change = ...

**Question 9**

If mRNA Expression (Affy) and mRNA Expression (RNAseq) were both in standard units, what would the regression parameters be?

Keep in mind, by definition, if a variable is in standard units then the mean is 0 and the standard deviation is 1.  Use Facts 1 and 2 about regression lines again.  (*Hint:* These calculations are actually less complicated.)


In [ ]:
intercept_in_su = ...
slope_in_su = ...


parameters_su = make_array(intercept_in_su, slope_in_su)
parameters_su

**Question 10**

The oldest and most commonly used cell line in Biology is the HeLa cell line, named after Henrietta Lacks, whose cervical cancer cells were taken without her consent in 1951 to create this cell line. The issue of data privacy and consent is very important to data science! You can read more about this topic [here](https://www.hopkinsmedicine.org/henriettalacks/).

The HeLa cell line is missing from our dataset. If we know that the HeLa mRNA Expression (Affy) value is 8.2, what is the predicted mRNA Expression (RNAseq) value? Use the values in `parameters` that we derived in Question 1.7, and assign the result to `hela_rnaseq`.



In [ ]:
hela_rnaseq = ...
hela_rnaseq

**Question 11**

Compute the predicted mRNA Expression (RNAseq) values from the mRNA Expression (Affy) values in the `pten` table. Use the values in the `parameters` array from Question 1.7, and assign the result to `predicted_rnaseq`. We'll plot your computed regression line with the scatter plot from after question 1.2 of mRNA Expression (Affy) vs. mRNA Expression (RNAseq).



In [ ]:
predicted_rnaseq = ...

# DON'T CHANGE/DELETE ANY OF THE BELOW CODE IN THIS CELL
(pten.with_column("Predicted mRNA Expression (RNAseq)", predicted_rnaseq)
 .select("mRNA Expression (Affy)", "mRNA Expression (RNAseq)", "Predicted mRNA Expression (RNAseq)")
 .scatter("mRNA Expression (Affy)"))
plt.ylabel("mRNA Expression (RNAseq)");

## Fitting a least-squares regression line

Recall that the least-square regression line is the unique straight line that minimizes root mean squared error (RMSE) among all possible fit lines. Using this property, we can find the equation of the regression line by finding the pair of slope and intercept values that minimize root mean squared error.

We define a function called `RMSE`. It takes two arguments:


1. the intercept of a line (a number).
2. the slope of a line (a number)


It returns a number that is the root mean squared error (RMSE) for a line defined with the arguments slope and intercept used to predict mRNA Expression (RNAseq) values from mRNA Expression (Affy) values for each row in the `pten` table.

*Hint: Errors are defined as the difference between the actual `y` values and the predicted `y` values.*

*Note: if you need a refresher on RMSE, here's the [link](https://www.inferentialthinking.com/chapters/15/3/Method_of_Least_Squares.html#Root-Mean-Squared-Error) from the textbook*


**Question 12**  Find the `RMSE` for the regression parameters you found in Question 7.  


$$RMSE = \sqrt{\frac{1}{n-2}\sum_{i=1}^n (y_i - \hat{y_i})^2}$$

$$MSE = \frac{1}{n-2}\sum_{i=1}^n (y_i - \hat{y_i})^2$$

$$SSE = \sum_{i=1}^n (y_i - \hat{y_i})^2$$


In [ ]:
def RMSE(intercept, slope):
    affy = pten.column("mRNA Expression (Affy)")
    rnaseq = pten.column("mRNA Expression (RNAseq)")
    predicted_rnaseq = slope*affy + intercept
    SE = (rnaseq - predicted_rnaseq)**2
    SSE = sum(SE)
    MSE = SSE/(len(affy) - 2)
    return MSE**0.5
    

# DON'T CHANGE THE FOLLOWING LINES BELOW IN THIS CELL
rmse_example = RMSE(0.5, 6)
rmse_example

In [ ]:
rmse_of_parameters = ...

**Question 13**

What is the RMSE of a line with slope 0 and intercept of the mean of `y` equal to?

*Hint 1: The line with slope 0 and intercept of mean of `y` is just a straight horizontal line at the mean of `y`*

*Hint 2: What does the formula for RMSE become if we input our predicted `y` values in the formula. Try writing it out on paper! It should be a familiar formula.*



*Write your answer here, replacing this text.*

**Question 14**

Find the parameters that minimizes RMSE of the regression line for mRNA Expression (Affy) vs. mRNA Expression (RNAseq). Assign the result to `minimized_parameters`.

If you haven't tried to use the `minimize` [function](http://data8.org/sp20/python-reference.html) yet, now is a great time to practice. Here's an [example from the textbook](https://www.inferentialthinking.com/chapters/15/3/Method_of_Least_Squares.html#numerical-optimization).

*Hint: Use the `RMSE` function in Question 1.12*

**NOTE: When you use the minimize function, please pass in `smooth=True` as the second argument to this function. You'll need to do this, otherwise, your answer will be incorrect**



In [ ]:
minimized_parameters = minimize(RMSE, smooth=True)
minimized_parameters

**Question 15**

The slope and intercept pair you found in Question 1.15 should be very similar to the values that you found in Question 1.7. Why were we able to minimize RMSE to find the same slope and intercept from the previous formulas?



*Write your answer here, replacing this text.*

**Question 16**

If we had instead minimized mean squared error (MSE), would we have gotten the same slope and intercept of the minimized root mean squared error (RMSE) results? Assign `same_parameters` to either `True` if you think yes, or `False` if you think no.




In [ ]:
same_parameters = ...
same_parameters

Let's look at the scatter plot of the relationship between mRNA Expression (Affy) and mRNA Expression (RNAseq) again:

In [ ]:
pten.scatter("mRNA Expression (Affy)", "mRNA Expression (RNAseq)", fit_line=True, facecolor ="red")

**Question 17**

Using a linear regression model, would we be able to obtain accurate predictions for most of the points? Explain why or why not.



*Write your answer here, replacing this text.*

## The `linregress` function in `scipy.stats`

What we did above was very interesting, and it helped us to better understand exactly *how* a linear regression analysis is done.  But in the future, you'll most likely use a build-in linear regression function.  One such function is the `linregress` command in `scipy.stats`.  Its use is demonstrated in the next set of questions.    

`import scipy.stats as stats`

`stats.linregress(x_array, y_array)`


### Garment Workers Data  

The [workers](https://archive.ics.uci.edu/ml/machine-learning-databases/00597/) data set describes the productivity of the workers, as a decimal between 0 and 1, and the financial incentive, in Bangladesh Taka (BDT), which is the national currency there.  

**Question 18**

Below is the code to run a linear regression analysis. This includes creating a scatterplot and putting the equation for the regression line along with the correlation on the graph.  

Does the graph seem to indicate that there is a positive or negative association/relationship between incentive and productivity?

1. Yes, there is a positive association; as incentive goes up so does productivity.

2. Yes, there is a negative association; as incentive goes up productivity goes down.

3. No, there is no indication of any sort of relationship or association.  


Put your answer in the cell below the graph.  

In [ ]:
workers = Table.read_table("garments_worker_productivity.csv")
workers = workers.select("incentive", "actual_productivity").where("incentive", are.between(1,500))

model_info = stats.linregress(workers.column("incentive"), workers.column("actual_productivity"))

slope = np.round(model_info[0],4)
intercept = np.round(model_info[1],2)
corr = np.round(model_info[2],3)

workers.scatter("incentive", "actual_productivity", fit_line=True)
plt.text(50, .5, f"Prod. = {slope}* Incent. +  {intercept}", color="red")
plt.text(62.5, .45,f"r = {corr}", color = "red")
plt.ylabel("Productivity")
plt.xlabel("Incentive (BDT)")
plt.title("Garment Workers Productivity vs Incentive");

In [ ]:
# Replace ... with 1,2 or 3

garment_answer = ...

**Question 19**  If the incentive was set to 130 one week, what productivity should we expect?  Put your answer in the cell below.  

In [ ]:
garment_prediction = ...

garment_prediction

## Chicken Pox in Hungary


**Question 20**

The cell below reads in and prepares a data set containing counts of number of children reporting that they have chicken pox in different counties in [Hungary](https://archive.ics.uci.edu/ml/machine-learning-databases/00580/).  

Copy and adapt the code from above to do the following:

    i. Run a linear regression analysis predicting the number of chicken pox cases in Pest from the number of 
    cases in Budapest.

    ii. Create a scatterplot comparing Pest to Budapest that also displays the regression line

    iii. Print the regression equation on the scatterplot

    iv. Print the correlation coefficient on the scatterplot

In [ ]:
pox = Table().read_table("hungary_chickenpox.csv")
pox = pox.where("BUDAPEST", are.below(120))
pox.show(3)


In [ ]:
## Run your linear regression analysis and make the scatterplot in this cell
model_info = ...



slope = np.round(model_info[0],4)
intercept = np.round(model_info[1],2)
corr = np.round(model_info[2],3)

pox.scatter("BUDAPEST", "PEST", fit_line=True)
plt.text(10, 200, f"Pest = {slope} Budapest +  {intercept}", color="red")
plt.text(20, 185, f"r = {corr}", color = "red")
plt.ylabel("Cases in Pest")
plt.xlabel("Cases in Budapest")
plt.title("Chicken Pox Cases in Hungary");

**Question 21**

Does you analysis seem to indicate that there may be an association between these number of chicken pox cases in Budapest and the same in Pest?

1. Yes, there is a positive association; the numbers of cases in both counties tends to rise and fall together.  

2. Yes, there is a negative association; as the number of cases in one county rises they tend to fall in the other county.  

3. No, there is no indication of any sort of relationship or association.  

Put your answer in the cell below.  

In [ ]:
# Replace ... with 1,2 or 3
pox_answer = ...

**Question 22**

In a week with 55 chicken pox cases in Budapest, about how many cases should we expect to see in Pest?

In [ ]:
pest_prediction = ...
pest_prediction

## Congratulations!

You're done with Homework 9.  Download it as HTML and submit it to the HW 9 folder in D2L. 